In [1]:
import numpy as np
import laspy as lp
import pylas
from matplotlib import pyplot as plt
from collections import Counter
from scipy.spatial import distance
import pandas as pd
import geopandas as gpd
from scipy.spatial import KDTree

# OPTIONAL! / Check the labels and what are they called

In [2]:
las_test = pylas.open('AHN.las')
data = las_test.read()
array_test = data.point_source_id
print("This las file has", len(data.points), "points")
data2 = data.point_format
print(data2.dimension_names)
counter_object3 = Counter(array_test)
keys3 = counter_object3.keys()
num_values3 = len(keys3)
print("This las file has", num_values3, "unique classifications")
# building = 6
# ground = 2
# others = 1
# water = 9

This las file has 1660906 points
('X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue')
This las file has 3 unique classifications


# 1. LOAD the data and Lower the point density

In [3]:
f = lp.file.File('AHN3.las', mode="r")
f2 = lp.file.File('AHN4.las', mode="r")

# concatenate the file coordinates
coord = np.c_[f.x, f.y, f.z, f.red, f.green, f.blue, f.intensity, f.classification]
coord2 = np.c_[f2.x, f2.y, f2.z, f2.red, f2.green, f2.blue, f2.intensity, f2.classification]

# bring the points into a local coordinate system
# now we have smaller coordiante values to deal with

xi = (f.x).astype('int')
yi = (f.y).astype('int')
zi = (f.z).astype('int')
xi2 = (f2.x).astype('int')
yi2 = (f2.y).astype('int')
zi2 = (f2.z).astype('int')

# Reduce the data! get a flat voxel index ii
# lowers the number of points feed to the algorithm
ii = xi + yi * xi.max() + zi * yi.max() * xi.max()
_, sl = np.unique(ii, return_index=True)
coord = coord[sl, :]

#
ii2 = xi2 + yi2 * xi2.max() + zi2 * yi2.max() * xi2.max()
_, sl2 = np.unique(ii2, return_index=True)
coord2 = coord2[sl2, :]
print("Initially we had", len(f.x), len(f2.x), "points. But, after lowering the point denity, the number of remainig points is", len(coord), len(coord2)) 

Initially we had 10194918 17668119 points. But, after lowering the point denity, the number of remainig points is 1218875 1245790


# 2. Match the data from 2 different epoch

In [4]:
# Construct KDTree from array2
tree = KDTree(coord2[:, :2])

# Find the closest point in array2 to each point in array1
dist, ind = tree.query(coord[:, :2])

# Reorder array2 based on ind
array2_reordered = coord2[ind]


In [5]:
print(len(array2_reordered))
print(len(coord))

1218875
1218875


In [6]:
df1 = pd.DataFrame(coord, columns=['xAHN3', 'yAHN3', 'zAHN3', 'redAHN3', 'greenAHN3', 'blueAHN3', 'intensityAHN3', 'labelAHN3'])
df2 = pd.DataFrame(array2_reordered, columns=['x', 'y', 'z', 'red', 'green', 'blue', 'intensity', 'label'])

In [7]:
df3 = pd.DataFrame(
    np.column_stack([df1,df2]),
    columns=df1.columns.append(df2.columns)
)

In [8]:
new_gdf_2 = df3
# new_gdf_2

# 3. Generate the change_label target values

In [9]:
new_gdf_2["change_label"] = new_gdf_2.apply(lambda x: 1 if x["labelAHN3"] == 6 and x["label"] !=6 else 2 if x["label"] == 6 and x["labelAHN3"] != 6 else 0, axis = 1)

In [10]:
# new_gdf_2

# 4. Prepare and save the arrays for deep learning models

In [11]:
# get the data in an array format
data_array = new_gdf_2.values

In [12]:
# extract features and targets

ahn3_features = data_array[:, 2:7]  # z until intensity
ahn4_features = data_array[:, 10:15]  # z of second data until intensity of second data
targets = data_array[:, 16]  # change label

In [13]:
#  Subtract and add features from 2 epochs with each other to get a single input
result_substraction = ahn3_features - ahn4_features
result_addition = ahn3_features + ahn4_features

In [14]:
# Save the coordinates alongside the new merged features
ahn_train_data = np.c_[data_array[:, 0:3], result_addition, result_substraction]

In [15]:
# if you want to stack multiple training data into one to train algorithm with more than just one tile
# AHN_train_stack = np.vstack((train_data,train_data2))
# AHN_target_stack = np.vstack((targets,targets2))

In [16]:
# Save the arrays

np.save('ahn_train_data.npy', ahn_train_data)
np.save('targets.npy', targets)
# np.save('AHN_train_stack.npy', AHN_train_stack)
# np.save('AHN_target_stack.npy', AHN_target_stack)
